In [1]:
import sys
sys.path.append('../')
from setting import config_read

In [2]:
from mapping import ResourceMapper
from graph import *
from sparql import SPAQLConverter

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [3]:
config = config_read('../')
rm = ResourceMapper(config)
cgg = ConceptualGraphGenerator(config)
qgg = QueryGraphGenerator(config)
sc = SPAQLConverter(config)

# 2-1. Query Partitioning

In [16]:
# user_input = '가수 태연의 첫 데뷔곡은?'
# user_input = '뱅뱅뱅 부른 음악그룹의 소속사는?'
# user_input = '용감한 형제가 작곡한 노래의 장르는?'
# user_input = '작곡가이며 가수인 사람'
# user_input = '장르가 발라드인 노래'
# user_input = '가수 버벌진트가 피처링한 노래'
# user_input = '태연이 처음 부른 노래는?'
# user_input = '소녀시대의 리더는?'
# user_input = '제니의 소속 그룹'
# user_input = '레인을 작곡한 사람'
user_input = '국적이 뉴질랜드 사람은?'
# user_input = 'AOA 부른 발라드 노래'
# user_input = '세월이 가면의 가수는?'
# user_input = 'AOA 발라드 노래?'
# user_input = '지드래곤 노래'
query_terms = rm.tokenize_query(user_input)
query_terms

['국적', '뉴질랜드', '사람']

In [17]:
dependent_idx = -1
for t_idx, token in enumerate(query_terms):
    if token in ['무슨', '어느', '어떻다', '몇']:
        dependent_idx = t_idx 
        break

if dependent_idx >= 0:
    new_query_terms = query_terms[:dependent_idx] + query_terms[dependent_idx+2:] + [query_terms[dependent_idx+1]]
else:
    new_query_terms = query_terms

In [18]:
new_query_terms

['국적', '뉴질랜드', '사람']

In [19]:
query_patitions = rm.partition_query(new_query_terms)
query_patitions

[['국적 뉴질랜드 사람'], ['국적', '뉴질랜드 사람'], ['국적 뉴질랜드', '사람'], ['국적', '뉴질랜드', '사람']]

# 2-2. Partition Scoring

In [20]:
p_star, score_list = rm.score_partition(query_patitions)
p_star

['국적', '뉴질랜드', '사람']

# 2-3. Generation of resource combination

In [21]:
resource_combinations = rm.combinate_resource(p_star)
resource_combinations

[('schema:nationality', 'skmo:뉴질랜드', 'foaf:Person')]

# 3. Conceptual graph generation

In [22]:
conceptual_graph = cgg.generate_conceptual_graph(resource_combinations)
conceptual_graph

[[('owl:Thing', 'schema:nationality', 'skmo:뉴질랜드'),
  ('skmo:뉴질랜드', 'Any P', 'foaf:Person')],
 [('owl:Thing', 'schema:nationality', 'foaf:Person'),
  ('skmo:뉴질랜드', 'Any P', 'foaf:Person')]]

# 4. Query graph generation

In [23]:
shortest_path_dict = qgg.search_at_tbox_level(conceptual_graph)
shortest_path_dict

{('owl:Thing', 'schema:nationality', 'skmo:뉴질랜드'): [],
 ('skmo:뉴질랜드',
  'Any P',
  'foaf:Person'): [(0.0,
   [('foaf:Person', 'schema:nationality', 'schema:Country(skmo:뉴질랜드)')])],
 ('owl:Thing',
  'schema:nationality',
  'foaf:Person'): [(0.0,
   [('foaf:Person', 'schema:nationality', 'schema:Country'),
    ('schema:Country', 'rdfs:subClassOf', 'owl:Thing')])]}

In [24]:
query_graph = qgg.generate_query_graph(conceptual_graph, shortest_path_dict)
query_graph

[(0.6666666666666666,
  [('foaf:Person', 'schema:nationality', 'schema:Country'),
   ('schema:Country', 'rdfs:subClassOf', 'owl:Thing'),
   ('foaf:Person', 'schema:nationality', 'schema:Country(skmo:뉴질랜드)')])]

# 5. SPARQL Conversion

In [25]:
final_query_graph = sc.select_query_graph(query_graph)
final_query_graph

[('foaf:Person', 'schema:nationality', 'schema:Country'),
 ('schema:Country', 'rdfs:subClassOf', 'owl:Thing'),
 ('foaf:Person', 'schema:nationality', 'schema:Country(skmo:뉴질랜드)')]

In [26]:
relation_triples = sc.extract_relation_triples(final_query_graph)
relation_triples

[('foaf:Person', 'schema:nationality', 'schema:Country'),
 ('foaf:Person', 'schema:nationality', 'schema:Country(skmo:뉴질랜드)')]

In [27]:
print('<USER Query>')
print(user_input)
print()


print('<SPARQL Query>')
knows_query = sc.generate_query(relation_triples)
print(knows_query)


print('\n<Answer>')
result = sc.excute_query(knows_query)
sc.print_result(result)

<USER Query>
국적이 뉴질랜드 사람은?

<SPARQL Query>
SELECT DISTINCT ?target 
WHERE
{
    ?target schema:nationality ?x0; schema:nationality skmo:뉴질랜드
}

<Answer>
로제  